In [11]:

import cv2
import os
import numpy as np

# Ensure contrib is available
assert hasattr(cv2, 'face'), "Please install opencv-contrib-python before running this."

dataset_path = "objectDetection/dataset"
faces = []
ids = []
user_id = 1  

for img_name in os.listdir(dataset_path):
    img_path = os.path.join(dataset_path, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue  # skip unreadable files
    faces.append(img)
    ids.append(user_id)

ids = np.array(ids)

clf = cv2.face.LBPHFaceRecognizer_create()
clf.train(faces, ids)

save_path = "objectDetection/classifier.yml"
clf.write(save_path)

print(f"Training complete! Model saved at: {os.path.abspath(save_path)}")

 

Training complete! Model saved at: C:\Users\HP\objectDetection\objectDetection\classifier.yml


In [18]:
import cv2
import os
import numpy as np

# Constants
dataset_path = "objectDetection/dataset"
model_save_path = "objectDetection/classifier.yml"
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
num_samples = 50

# Step 1: Capture faces for a new person
person_name = input("Enter person name: ").strip()
person_folder = os.path.join(dataset_path, person_name)
os.makedirs(person_folder, exist_ok=True)

cap = cv2.VideoCapture(0)
count = 0

print(f"Starting to capture {num_samples} face samples for {person_name}...")

while True:
    ret, frame = cap.read()
    if not ret:
        continue
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    for (x, y, w, h) in faces:
        count += 1
        face_img = gray[y:y+h, x:x+w]
        img_path = os.path.join(person_folder, f"{person_name}_{count}.jpg")
        cv2.imwrite(img_path, face_img)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 255), 2)
        cv2.putText(frame, f"Sample {count}/{num_samples}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        if count >= num_samples:
            break
            
    cv2.imshow("Capture Faces", frame)
    if cv2.waitKey(1) & 0xFF == ord('q') or count >= num_samples:
        break

cap.release()
cv2.destroyAllWindows()

print(f"Captured {count} face images for {person_name} in folder '{person_folder}'")

# Step 2: Prepare dataset and train face recognizer on all data
faces = []
labels = []
label_dict = {}
label_id = 0

for person in os.listdir(dataset_path):
    person_dir = os.path.join(dataset_path, person)
    if not os.path.isdir(person_dir):
        continue
    label_dict[label_id] = person
    for img_name in os.listdir(person_dir):
        img_path = os.path.join(person_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces_rects = face_cascade.detectMultiScale(gray, 1.2, 5)
        for (x, y, w, h) in faces_rects:
            faces.append(gray[y:y+h, x:x+w])
            labels.append(label_id)
    label_id += 1

labels = np.array(labels)
clf = cv2.face.LBPHFaceRecognizer_create()
clf.train(faces, labels)
clf.write(model_save_path)

print(f"Training complete! Model saved at: {model_save_path}")
print("Label IDs and corresponding persons:")
for key, val in label_dict.items():
    print(f"{key} : {val}")


Enter person name:  mathee


Starting to capture 50 face samples for mathee...
Captured 50 face images for mathee in folder 'objectDetection/dataset\mathee'
Training complete! Model saved at: objectDetection/classifier.yml
Label IDs and corresponding persons:
0 : Ishini
1 : ivar
2 : mathee
